In [1]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [3]:
get_2693_sample_df = pd.read_excel('../mark_csv/revise5550_JYY0315_importantdata.xlsx')[['new_label', 'revised_Truth', '處置其他結束（描述）']]
context_split_csv = get_2693_sample_df.rename(columns={'new_label':'label'})
print(context_split_csv.shape)
positive_context_split_csv = (context_split_csv[context_split_csv.revised_Truth=='Y'])['處置其他結束（描述）'].values.tolist()
print(len(positive_context_split_csv))


concat1 = pd.read_csv('/ssd8/chih/project/yadong/process_csv/PeriHD_not_markdata-sampe2000.csv')
concat2 = pd.read_csv('/ssd8/chih/project/yadong/mark_csv/train_peri_mark_sample_693_JYY.csv')
peri_filter_dataset = pd.concat([concat1, concat2])
# peri_filter_dataset = peri_filter_dataset['處置其他結束（描述）']
# peri_693_dataset = pd.read_csv('../mark_csv/train_peri_mark_sample_693_JYY.csv')[['truth', '處置其他結束（描述）']]
# peri_filter_dataset = peri_693_dataset[peri_693_dataset['truth']=='Y']
print(peri_filter_dataset.shape)

(5549, 3)
666
(2693, 3)


In [8]:
# 先切割標記後的資料以確保訓練和驗證資料無重疊
keyword_col = []
for seq in peri_filter_dataset['處置其他結束（描述）'].values.tolist(): #逐筆搜尋pos
    seq_sp = str(seq).replace("。","，").replace(",","，").replace("\n","").split('，')
    temp_keyword = ''
    for pos in positive_context_split_csv:
        if str(pos) in seq_sp:
            temp_keyword = temp_keyword + str(pos)+'，'
    if len(temp_keyword)<1:
        temp_keyword = 'Negative_Sequence'
    keyword_col.append(temp_keyword)
    # break

In [10]:
peri_filter_dataset['KeyWord_dict'] = keyword_col
peri_filter_dataset = peri_filter_dataset[['處置其他結束（描述）', 'KeyWord_dict']]
peri_filter_dataset['label'] = [0 if i =='Negative_Sequence' else 1 for i in peri_filter_dataset.KeyWord_dict.tolist()]
data_train, data_test = train_test_split(peri_filter_dataset, test_size=0.1, random_state=42, stratify=peri_filter_dataset['label'])

In [11]:
data_train

,處置其他結束（描述）,KeyWord_dict,label
527,HD畢，頭痛情形有較改善但還是會痛，個知若返家仍不適需就醫，病人了解，透析後跌捯評估0分，A...,Negative_Sequence,0
145,病人由L''''t Femoral DLC HD，exit site clear，out針反...,Negative_Sequence,0
1306,HD畢，無不適，AVF止血後紗布覆蓋，透析後防跌評估為0項，家屬陪同協助坐輪椅離開HDR。,Negative_Sequence,0
516,透析畢，透析後體重與乾體重誤差0.7公斤，病人表示透析中有吃東西，現不適情形已較緩解，PER...,現不適情形已較緩解，,1
110,病人意識清楚，於0740由傳送及看護陪同推床入HDR，透析前體重76.3公斤、血壓129/7...,Negative_Sequence,0
...,...,...,...
643,HD畢，無不適情形，R't perm cath on hepain lock留置，由外傭協助...,Negative_Sequence,0
80,ahd give B-complex 4ml iv f/u shount three2+ g...,Negative_Sequence,0
447,HD畢，無不適情形，AVF已止血紗布覆蓋，家屬陪同返家。,Negative_Sequence,0
758,"HF clear, AVG thrill++, go home",Negative_Sequence,0


In [ ]:
temp = ['dferdderefdfd']

In [40]:
merage_sentence = []
for seq in range(len(data_test)):
    if True:
        rep_seq = data_test.iloc[seq]['處置其他結束（描述）']
        rep_seq = str(rep_seq).replace("。","，").replace(",","，").replace("\n","").replace(";","，").replace("；","，").split('，')
        key_word = data_test.iloc[seq]['KeyWord_dict']
        key_word = str(key_word).replace("。","，").replace(",","，").replace("\n","").replace(";","，").replace("；","，").split('，')
        key_word = [x for x in key_word if x != ""]
        if len(rep_seq)!=1:
            for i in range(len(rep_seq)-1):
                new_sentence = rep_seq[i]+"，"+rep_seq[i+1]
                if (rep_seq[i] in key_word) or (rep_seq[i+1] in key_word):
                    merage_sentence.append(['Y',new_sentence])
                elif len(rep_seq[i+1])>0:
                    merage_sentence.append(['N',new_sentence])
        # else:
        #     print('pass')
        #     if rep_seq in key_word:
        #         merage_sentence.append(['Y-pass',new_sentence])
        #     else:
        #         merage_sentence.append(['N-pass',new_sentence])

In [41]:
(pd.DataFrame(merage_sentence, columns=['Truth', '處置其他結束（描述）']).drop_duplicates(subset=['處置其他結束（描述）'])).to_csv("./dataset/PeriHD_Sample-2693_Sentence-merge_internal-test[pos+neg].csv", encoding='utf-8-sig', index=False)

In [42]:
merage_sentence = []
for seq in range(len(data_train)):
    rep_seq = data_train.iloc[seq]['處置其他結束（描述）']
    rep_seq = str(rep_seq).replace("。","，").replace(",","，").replace("\n","").replace(";","，").replace("；","，").split('，')
    key_word = data_train.iloc[seq]['KeyWord_dict']
    key_word = str(key_word).replace("。","，").replace(",","，").replace("\n","").replace(";","，").replace("；","，").split('，')
    key_word = [x for x in key_word if x != ""]
    if len(rep_seq)!=1:
        for i in range(len(rep_seq)-1):
            new_sentence = rep_seq[i]+"，"+rep_seq[i+1]
            if (rep_seq[i] in key_word) or (rep_seq[i+1] in key_word):
                merage_sentence.append(['Y',new_sentence])
            elif len(rep_seq[i+1])>0:
                merage_sentence.append(['N',new_sentence])
    # if seq==10:
    #     break

In [43]:
len(merage_sentence)

16805

In [45]:
(pd.DataFrame(merage_sentence, columns=['Truth', '處置其他結束（描述）']).drop_duplicates(subset=['處置其他結束（描述）'])).to_csv("./dataset/PeriHD_Sample-2693_Sentence-merge_internal-train[pos+neg].csv", encoding='utf-8-sig', index=False)

In [46]:
get_1124_sample_df = pd.read_csv("./dataset/train_intra_mark_sample_1124_JYY.csv")[['truth','處置其他+症狀處置（描述）']]
get_1124_sample_df = get_1124_sample_df.rename(columns={'truth':'label'})
get_1124_sample_df = get_1124_sample_df.dropna()
print(get_1124_sample_df.shape)
data_train, data_test = train_test_split(get_1124_sample_df, test_size=0.1, random_state=42, stratify=get_1124_sample_df['label'])
data_train.to_csv("./dataset/IntraHD_Sample-1124_Sentence-merge_internal-train.csv", encoding='utf-8-sig', index=False)
data_test.to_csv("./dataset/IntraHD_Sample-1124_Sentence-merge_internal-test.csv", encoding='utf-8-sig', index=False)
print(data_train.shape, data_test.shape)

(1121, 2)
(1008, 2) (113, 2)


In [20]:
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_columns', 20)
# data_test